In [142]:
from database_wrapper import DatabaseWrapper
from sreality_scraper.sreality.spiders.sreality_spider import SrealityUrlBuilder
import pandas as pd
import numpy as np

db = DatabaseWrapper("listings.db")
df = db.get_df()
db.close_conn()
df

,address,area,available_from,balcony,cellar,created,description,disposition,elevator,floor,...,pets,public_transport,rent,security_deposit,service_fees,status,terrace,type,updated,url
0,"Košická, Praha 10 - Vršovice",49.0,Ihned,0,0,2024-03-15 19:46:48.984998,nabízíme vám k pronájmu útulný byt 1+kk o veli...,2,1,5. podlaží z celkem 5,...,None,None,19900,None,None,Po rekonstrukci,0,3,2024-03-15 19:49:48.122517,https://www.sreality.cz/detail/pronajem/byt/1+...
1,"Peroutkova, Praha 5 - Jinonice",24.0,15.04.2024,0,1,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,2,1,2. podlaží,...,None,None,13990,None,+ poplatky za služby vč. elektřiny 3.633 Kč/měsíc,Velmi dobrý,0,3,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/1+...
2,"Za Hládkovem, Praha 6 - Střešovice",75.0,Ihned,1,0,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,5,1,4. podlaží,...,None,None,25490,None,"+ poplatky 3.550 Kč/měsíc + elektřina, která b...",Velmi dobrý,0,2,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/2+...
3,"Pelušková, Praha 9 - Kyje",40.0,Ihned,1,1,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,4,1,1. podlaží,...,None,None,13990,None,"+poplatky 3.080 Kč/měsíc + elektřina a plyn, k...",Velmi dobrý,0,2,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/2+...
4,"Na Břehu, Praha 9 - Vysočany",67.0,01.05.2024,1,0,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,4,1,2. podlaží,...,None,None,22490,None,"+ poplatky 1.822 Kč/měsíc + elektřina, která b...",Velmi dobrý,0,2,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/2+...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,"Bartákova, Praha 4 - Krč",37.0,Ihned,0,0,2024-03-16 11:57:33.477657,pronájem bytu 1+kk 34m2 s lodžií v ulici bartá...,2,1,4. podlaží z celkem 8,...,None,None,12500,None,+ 2500 zúčtovatelné poplatky,Velmi dobrý,0,1,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/1+...
5276,"Šenácká, Praha 4 - Hodkovičky",37.0,20.03.2024,1,1,2024-03-16 11:57:33.477657,nový byt 1+kk (37 m2) s terasou (8 m2) ve 2.pa...,2,1,3. podlaží,...,None,None,16000,None,"+ poplatky 2.600,- Kč/měs. + spotřeba elektřiny",Novostavba,1,2,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/1+...
5277,"Opletalova, Praha 1 - Nové Město",43.0,25.03.2024,0,0,2024-03-16 11:57:33.477657,zařízený byt 2+kk po kompletní rekonstrukci o ...,4,1,4. podlaží z celkem 6,...,None,None,28500,None,+ poplatky vč. energií a internetu 5.500Kč/měs.,Po rekonstrukci,0,2,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/2+...
5278,"Služská, Praha 8 - Kobylisy",85.0,Ihned,1,0,2024-03-16 11:57:33.477657,pronájem bytu v praze na kobylisich (ul. sluzs...,7,0,2. podlaží,...,None,None,27500,None,+ 1500 zálohy energie,Velmi dobrý,0,3,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/3+...


In [143]:
# set id as index
df.set_index('id', inplace=True)

In [144]:
# print unique disposition values
print(df["disposition"].unique())
dispositions = df["disposition"].unique()

[2 5 4 11 9 47 6 10 7 3 8 16 '1+kk' '2+kk' '4+1' '1+1' '2+1' '3+1'
 'Garsoniéra' '4+kk' None 12 '5+1' '3+kk' '5+kk' 'Ostatní']


In [145]:
# for each unique disposition value try running the map_category_sub_cb from SrealityUrlBuilder
for disposition in dispositions:
    if not disposition or isinstance(disposition, str):
        continue

    if isinstance(disposition, int):    
        df['disposition'] = df['disposition'].replace(disposition, SrealityUrlBuilder.map_category_sub_cb(disposition))


        

In [146]:
df['disposition'].unique()

array(['1+kk', '2+1', '2+kk', '5+1', '4+1', 'pokoj', '3+kk', '5+kk',
       '3+1', '1+1', '4+kk', 'atypicky', 'Garsoniéra', None, '6-a-vice',
       'Ostatní'], dtype=object)

In [147]:
df['disposition'] = df['disposition'].replace('Garsoniéra', '1+kk') # bezrealitky specific
df['disposition'] = df['disposition'].replace('Ostatní', 'other') # bezrealitky specific
df['disposition'] = df['disposition'].replace('atypicky', 'other') # sreality specific
df['disposition'] = df['disposition'].replace('pokoj', 'other') # sreality specific
df['disposition'] = df['disposition'].replace('6+kk', '6-a-více')
df['disposition'] = df['disposition'].replace('6+1', '6-a-více')
df['disposition'] = df['disposition'].replace('7+kk', '6-a-více')
df['disposition'] = df['disposition'].replace('7+1', '6-a-více')


In [148]:
df['disposition'].sort_values().unique()

array(['1+1', '1+kk', '2+1', '2+kk', '3+1', '3+kk', '4+1', '4+kk', '5+1',
       '5+kk', '6-a-vice', 'other', None], dtype=object)

In [149]:
df

,address,area,available_from,balcony,cellar,created,description,disposition,elevator,floor,...,pets,public_transport,rent,security_deposit,service_fees,status,terrace,type,updated,url
id,,,,,,,,,,,,,,,,,,,,,
660215116,"Košická, Praha 10 - Vršovice",49.0,Ihned,0,0,2024-03-15 19:46:48.984998,nabízíme vám k pronájmu útulný byt 1+kk o veli...,1+kk,1,5. podlaží z celkem 5,...,None,None,19900,None,None,Po rekonstrukci,0,3,2024-03-15 19:49:48.122517,https://www.sreality.cz/detail/pronajem/byt/1+...
2543723852,"Peroutkova, Praha 5 - Jinonice",24.0,15.04.2024,0,1,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,1+kk,1,2. podlaží,...,None,None,13990,None,+ poplatky za služby vč. elektřiny 3.633 Kč/měsíc,Velmi dobrý,0,3,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/1+...
3952227660,"Za Hládkovem, Praha 6 - Střešovice",75.0,Ihned,1,0,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,2+1,1,4. podlaží,...,None,None,25490,None,"+ poplatky 3.550 Kč/měsíc + elektřina, která b...",Velmi dobrý,0,2,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/2+...
3895641420,"Pelušková, Praha 9 - Kyje",40.0,Ihned,1,1,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,2+kk,1,1. podlaží,...,None,None,13990,None,"+poplatky 3.080 Kč/měsíc + elektřina a plyn, k...",Velmi dobrý,0,2,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/2+...
498279756,"Na Břehu, Praha 9 - Vysočany",67.0,01.05.2024,1,0,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,2+kk,1,2. podlaží,...,None,None,22490,None,"+ poplatky 1.822 Kč/měsíc + elektřina, která b...",Velmi dobrý,0,2,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/2+...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998067020,"Bartákova, Praha 4 - Krč",37.0,Ihned,0,0,2024-03-16 11:57:33.477657,pronájem bytu 1+kk 34m2 s lodžií v ulici bartá...,1+kk,1,4. podlaží z celkem 8,...,None,None,12500,None,+ 2500 zúčtovatelné poplatky,Velmi dobrý,0,1,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/1+...
2534937932,"Šenácká, Praha 4 - Hodkovičky",37.0,20.03.2024,1,1,2024-03-16 11:57:33.477657,nový byt 1+kk (37 m2) s terasou (8 m2) ve 2.pa...,1+kk,1,3. podlaží,...,None,None,16000,None,"+ poplatky 2.600,- Kč/měs. + spotřeba elektřiny",Novostavba,1,2,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/1+...
3320321356,"Opletalova, Praha 1 - Nové Město",43.0,25.03.2024,0,0,2024-03-16 11:57:33.477657,zařízený byt 2+kk po kompletní rekonstrukci o ...,2+kk,1,4. podlaží z celkem 6,...,None,None,28500,None,+ poplatky vč. energií a internetu 5.500Kč/měs.,Po rekonstrukci,0,2,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/2+...


In [150]:
# drop entries with area bigger than 1000
df = df[df['area'] < 1000]
df['area'].sort_values()


id
547824          6.0
3031377228      8.0
222887          9.0
3597452620     10.0
2750821708     10.0
              ...  
1672553804    301.0
2769237324    308.0
1763648844    386.0
3815060812    421.0
2434610508    435.0
Name: area, Length: 5278, dtype: float64

In [151]:
df.loc[:, 'available_from'] = df['available_from'].str.replace(' ', '')


In [152]:
# Get the indices of rows with invalid date format

invalid_indices = pd.to_datetime(df['available_from'], errors='coerce').isnull()
# get valid indices

df.loc[~invalid_indices, 'available_from'] = pd.to_datetime(df.loc[~invalid_indices, 'available_from'], format='%d.%m.%Y', errors='coerce')
df.loc[invalid_indices, 'available_from'].unique()



/tmp/ipykernel_21374/3872875631.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  invalid_indices = pd.to_datetime(df['available_from'], errors='coerce').isnull()


array(['Ihned', None], dtype=object)

In [153]:
df.loc[df['available_from'] == 'Ihned', 'available_from'] = pd.to_datetime(df.loc[df['available_from'] == 'Ihned', 'updated'], format="%Y-%m-%d %H:%M:%S.%f", errors='coerce')


In [154]:
# verify available_from column is in datetime format
df['available_from'] = pd.to_datetime(df['available_from'], errors='coerce')
df['available_from']

/tmp/ipykernel_21374/1473313099.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['available_from'] = pd.to_datetime(df['available_from'], errors='coerce')


id
660215116    2024-03-15 19:49:48.122517
2543723852   2024-04-15 00:00:00.000000
3952227660   2024-03-15 19:46:48.984998
3895641420   2024-03-15 19:46:48.984998
498279756    2024-05-01 00:00:00.000000
                        ...            
1998067020   2024-03-16 11:57:33.477657
2534937932   2024-03-20 00:00:00.000000
3320321356   2024-03-25 00:00:00.000000
3085440332   2024-03-16 11:57:33.477657
3286766924   2024-03-16 12:10:28.660393
Name: available_from, Length: 5278, dtype: datetime64[ns]

In [155]:
df

,address,area,available_from,balcony,cellar,created,description,disposition,elevator,floor,...,pets,public_transport,rent,security_deposit,service_fees,status,terrace,type,updated,url
id,,,,,,,,,,,,,,,,,,,,,
660215116,"Košická, Praha 10 - Vršovice",49.0,2024-03-15 19:49:48.122517,0,0,2024-03-15 19:46:48.984998,nabízíme vám k pronájmu útulný byt 1+kk o veli...,1+kk,1,5. podlaží z celkem 5,...,None,None,19900,None,None,Po rekonstrukci,0,3,2024-03-15 19:49:48.122517,https://www.sreality.cz/detail/pronajem/byt/1+...
2543723852,"Peroutkova, Praha 5 - Jinonice",24.0,2024-04-15 00:00:00.000000,0,1,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,1+kk,1,2. podlaží,...,None,None,13990,None,+ poplatky za služby vč. elektřiny 3.633 Kč/měsíc,Velmi dobrý,0,3,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/1+...
3952227660,"Za Hládkovem, Praha 6 - Střešovice",75.0,2024-03-15 19:46:48.984998,1,0,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,2+1,1,4. podlaží,...,None,None,25490,None,"+ poplatky 3.550 Kč/měsíc + elektřina, která b...",Velmi dobrý,0,2,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/2+...
3895641420,"Pelušková, Praha 9 - Kyje",40.0,2024-03-15 19:46:48.984998,1,1,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,2+kk,1,1. podlaží,...,None,None,13990,None,"+poplatky 3.080 Kč/měsíc + elektřina a plyn, k...",Velmi dobrý,0,2,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/2+...
498279756,"Na Břehu, Praha 9 - Vysočany",67.0,2024-05-01 00:00:00.000000,1,0,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,2+kk,1,2. podlaží,...,None,None,22490,None,"+ poplatky 1.822 Kč/měsíc + elektřina, která b...",Velmi dobrý,0,2,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/2+...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998067020,"Bartákova, Praha 4 - Krč",37.0,2024-03-16 11:57:33.477657,0,0,2024-03-16 11:57:33.477657,pronájem bytu 1+kk 34m2 s lodžií v ulici bartá...,1+kk,1,4. podlaží z celkem 8,...,None,None,12500,None,+ 2500 zúčtovatelné poplatky,Velmi dobrý,0,1,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/1+...
2534937932,"Šenácká, Praha 4 - Hodkovičky",37.0,2024-03-20 00:00:00.000000,1,1,2024-03-16 11:57:33.477657,nový byt 1+kk (37 m2) s terasou (8 m2) ve 2.pa...,1+kk,1,3. podlaží,...,None,None,16000,None,"+ poplatky 2.600,- Kč/měs. + spotřeba elektřiny",Novostavba,1,2,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/1+...
3320321356,"Opletalova, Praha 1 - Nové Město",43.0,2024-03-25 00:00:00.000000,0,0,2024-03-16 11:57:33.477657,zařízený byt 2+kk po kompletní rekonstrukci o ...,2+kk,1,4. podlaží z celkem 6,...,None,None,28500,None,+ poplatky vč. energií a internetu 5.500Kč/měs.,Po rekonstrukci,0,2,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/2+...


In [156]:
df['balcony'].unique()

array([0, 1, None, 'Balkón 6 m²', 'Balkón 4 m²', 'Balkón 1,1 m²',
       'Balkón 2 m²', 'Balkón 5,3 m²', 'Balkón', 'Balkón 8 m²',
       'Balkón 5 m²', 'Balkón 5,5 m²', 'Balkón 3 m²', 'Balkón 5,6 m²',
       'Balkón 4,1 m²', 'Balkón 4,7 m²', 'Balkón 10 m²', 'Balkón 15 m²',
       'Balkón 4,86 m²', 'Balkón 3,6 m²', 'Balkón 12 m²', 'Balkón 3,1 m²',
       'Balkón 7 m²', 'Balkón 3,45 m²', 'Balkón 8,1 m²', 'Balkón 9 m²',
       'Balkón 14 m²', 'Balkón 4,2 m²', 'Balkón 16 m²', 'Balkón 18 m²',
       'Balkón 3,5 m²', 'Balkón 8,5 m²', 'Balkón 1,5 m²', 'Balkón 2,5 m²',
       'Balkón 1,6 m²', 'Balkón 12,1 m²', 'Balkón 20 m²', 'Balkón 6,8 m²',
       'Balkón 7,2 m²', 'Balkón 3,2 m²', 'Balkón 13,1 m²', 'Balkón 31 m²',
       'Balkón 13 m²', 'Balkón 5,8 m²', 'Balkón 11 m²', 'Balkón 19 m²',
       'Balkón 4,02 m²', 'Balkón 4,8 m²', 'Balkón 4,6 m²',
       'Balkón 3,14 m²', 'Balkón 5,1 m²', 'Balkón 7,3 m²',
       'Balkón 2,25 m²', 'Balkón 3,4 m²', 'Balkón 5,4 m²',
       'Balkón 5,79 m²', 'Balkón 

In [157]:
df['balcony'] = df['balcony'].replace('Balk.*', 1, regex=True) # TODO: fix this, the replacement must be ran only on strings
df['balcony'] = df['balcony'].replace(np.nan, 0)
df['balcony'].unique()

/tmp/ipykernel_21374/3372007336.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['balcony'] = df['balcony'].replace('Balk.*', 1, regex=True) # TODO: fix this, the replacement must be ran only on strings
/tmp/ipykernel_21374/3372007336.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['balcony'] = df['balcony'].replace('Balk.*', 1, regex=True) # TODO: fix this, the replacement must be ran only on strings
/tmp/ipykernel_21374/3372007336.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

array([0., 1.])

In [158]:
df

,address,area,available_from,balcony,cellar,created,description,disposition,elevator,floor,...,pets,public_transport,rent,security_deposit,service_fees,status,terrace,type,updated,url
id,,,,,,,,,,,,,,,,,,,,,
660215116,"Košická, Praha 10 - Vršovice",49.0,2024-03-15 19:49:48.122517,0.0,0,2024-03-15 19:46:48.984998,nabízíme vám k pronájmu útulný byt 1+kk o veli...,1+kk,1,5. podlaží z celkem 5,...,None,None,19900,None,None,Po rekonstrukci,0,3,2024-03-15 19:49:48.122517,https://www.sreality.cz/detail/pronajem/byt/1+...
2543723852,"Peroutkova, Praha 5 - Jinonice",24.0,2024-04-15 00:00:00.000000,0.0,1,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,1+kk,1,2. podlaží,...,None,None,13990,None,+ poplatky za služby vč. elektřiny 3.633 Kč/měsíc,Velmi dobrý,0,3,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/1+...
3952227660,"Za Hládkovem, Praha 6 - Střešovice",75.0,2024-03-15 19:46:48.984998,1.0,0,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,2+1,1,4. podlaží,...,None,None,25490,None,"+ poplatky 3.550 Kč/měsíc + elektřina, která b...",Velmi dobrý,0,2,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/2+...
3895641420,"Pelušková, Praha 9 - Kyje",40.0,2024-03-15 19:46:48.984998,1.0,1,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,2+kk,1,1. podlaží,...,None,None,13990,None,"+poplatky 3.080 Kč/měsíc + elektřina a plyn, k...",Velmi dobrý,0,2,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/2+...
498279756,"Na Břehu, Praha 9 - Vysočany",67.0,2024-05-01 00:00:00.000000,1.0,0,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,2+kk,1,2. podlaží,...,None,None,22490,None,"+ poplatky 1.822 Kč/měsíc + elektřina, která b...",Velmi dobrý,0,2,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/2+...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998067020,"Bartákova, Praha 4 - Krč",37.0,2024-03-16 11:57:33.477657,0.0,0,2024-03-16 11:57:33.477657,pronájem bytu 1+kk 34m2 s lodžií v ulici bartá...,1+kk,1,4. podlaží z celkem 8,...,None,None,12500,None,+ 2500 zúčtovatelné poplatky,Velmi dobrý,0,1,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/1+...
2534937932,"Šenácká, Praha 4 - Hodkovičky",37.0,2024-03-20 00:00:00.000000,1.0,1,2024-03-16 11:57:33.477657,nový byt 1+kk (37 m2) s terasou (8 m2) ve 2.pa...,1+kk,1,3. podlaží,...,None,None,16000,None,"+ poplatky 2.600,- Kč/měs. + spotřeba elektřiny",Novostavba,1,2,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/1+...
3320321356,"Opletalova, Praha 1 - Nové Město",43.0,2024-03-25 00:00:00.000000,0.0,0,2024-03-16 11:57:33.477657,zařízený byt 2+kk po kompletní rekonstrukci o ...,2+kk,1,4. podlaží z celkem 6,...,None,None,28500,None,+ poplatky vč. energií a internetu 5.500Kč/měs.,Po rekonstrukci,0,2,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/2+...


In [159]:
df['cellar'] = df['cellar'].replace('Sklep.*', 1, regex=True)
df['cellar'] = df['cellar'].replace(np.nan, 0)
df

/tmp/ipykernel_21374/1473772411.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['cellar'] = df['cellar'].replace('Sklep.*', 1, regex=True)
/tmp/ipykernel_21374/1473772411.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cellar'] = df['cellar'].replace('Sklep.*', 1, regex=True)
/tmp/ipykernel_21374/1473772411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

,address,area,available_from,balcony,cellar,created,description,disposition,elevator,floor,...,pets,public_transport,rent,security_deposit,service_fees,status,terrace,type,updated,url
id,,,,,,,,,,,,,,,,,,,,,
660215116,"Košická, Praha 10 - Vršovice",49.0,2024-03-15 19:49:48.122517,0.0,0.0,2024-03-15 19:46:48.984998,nabízíme vám k pronájmu útulný byt 1+kk o veli...,1+kk,1,5. podlaží z celkem 5,...,None,None,19900,None,None,Po rekonstrukci,0,3,2024-03-15 19:49:48.122517,https://www.sreality.cz/detail/pronajem/byt/1+...
2543723852,"Peroutkova, Praha 5 - Jinonice",24.0,2024-04-15 00:00:00.000000,0.0,1.0,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,1+kk,1,2. podlaží,...,None,None,13990,None,+ poplatky za služby vč. elektřiny 3.633 Kč/měsíc,Velmi dobrý,0,3,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/1+...
3952227660,"Za Hládkovem, Praha 6 - Střešovice",75.0,2024-03-15 19:46:48.984998,1.0,0.0,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,2+1,1,4. podlaží,...,None,None,25490,None,"+ poplatky 3.550 Kč/měsíc + elektřina, která b...",Velmi dobrý,0,2,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/2+...
3895641420,"Pelušková, Praha 9 - Kyje",40.0,2024-03-15 19:46:48.984998,1.0,1.0,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,2+kk,1,1. podlaží,...,None,None,13990,None,"+poplatky 3.080 Kč/měsíc + elektřina a plyn, k...",Velmi dobrý,0,2,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/2+...
498279756,"Na Břehu, Praha 9 - Vysočany",67.0,2024-05-01 00:00:00.000000,1.0,0.0,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,2+kk,1,2. podlaží,...,None,None,22490,None,"+ poplatky 1.822 Kč/měsíc + elektřina, která b...",Velmi dobrý,0,2,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/2+...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998067020,"Bartákova, Praha 4 - Krč",37.0,2024-03-16 11:57:33.477657,0.0,0.0,2024-03-16 11:57:33.477657,pronájem bytu 1+kk 34m2 s lodžií v ulici bartá...,1+kk,1,4. podlaží z celkem 8,...,None,None,12500,None,+ 2500 zúčtovatelné poplatky,Velmi dobrý,0,1,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/1+...
2534937932,"Šenácká, Praha 4 - Hodkovičky",37.0,2024-03-20 00:00:00.000000,1.0,1.0,2024-03-16 11:57:33.477657,nový byt 1+kk (37 m2) s terasou (8 m2) ve 2.pa...,1+kk,1,3. podlaží,...,None,None,16000,None,"+ poplatky 2.600,- Kč/měs. + spotřeba elektřiny",Novostavba,1,2,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/1+...
3320321356,"Opletalova, Praha 1 - Nové Město",43.0,2024-03-25 00:00:00.000000,0.0,0.0,2024-03-16 11:57:33.477657,zařízený byt 2+kk po kompletní rekonstrukci o ...,2+kk,1,4. podlaží z celkem 6,...,None,None,28500,None,+ poplatky vč. energií a internetu 5.500Kč/měs.,Po rekonstrukci,0,2,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/2+...


In [160]:
df['elevator'].unique()

array([1, 2, 0, 'Výtah', None], dtype=object)

In [161]:
df[df['elevator'] == 2]

,address,area,available_from,balcony,cellar,created,description,disposition,elevator,floor,...,pets,public_transport,rent,security_deposit,service_fees,status,terrace,type,updated,url
id,,,,,,,,,,,,,,,,,,,,,
2052994380,"Hroznová, Praha 1 - Malá Strana",130.0,2024-03-15 19:46:48.984998,0.0,1.0,2024-03-15 19:46:48.984998,vzhledem k velmi vysokému zájmu o námi nabízen...,5+1,2,4. podlaží,...,None,None,37990,None,"+ poplatky 3.599 Kč/měsíc + elektřina a plyn, ...",Dobrý,0,2,2024-03-15 19:46:48.984998,https://www.sreality.cz/detail/pronajem/byt/5+...
3615040844,"U Havlíčkových sadů, Praha 2 - Vinohrady",141.0,2024-03-15 21:21:43.174528,1.0,0.0,2024-03-15 21:21:43.174528,k nastěhování ihned.<br />nabízíme vám k proná...,4+1,2,3. podlaží,...,None,None,55000,None,None,Velmi dobrý,0,2,2024-03-15 21:21:43.174528,https://www.sreality.cz/detail/pronajem/byt/4+...
3799590220,"Petřínská, Praha 5 - Malá Strana",55.0,2024-03-15 21:21:43.174528,0.0,0.0,2024-03-15 21:21:43.174528,ve výhradním zastoupení majitele nabízíme k po...,2+1,2,3. podlaží z celkem 5,...,None,None,16500,None,None,Dobrý,0,2,2024-03-15 21:21:43.174528,https://www.sreality.cz/detail/pronajem/byt/2+...
2223170892,"Nuselská, Praha 4 - Michle",44.0,2024-03-15 21:21:43.174528,0.0,1.0,2024-03-15 21:21:43.174528,k dlouhodobému podnájmu nabízíme prostorný byt...,2+kk,2,5. podlaží z celkem 6,...,None,None,15000,None,"vratná kauce Kč, náklady na bydlení 4 000 Kč v...",Velmi dobrý,0,2,2024-03-15 21:21:43.174528,https://www.sreality.cz/detail/pronajem/byt/2+...
1437607244,"Bořivojova, Praha 3 - Vinohrady",138.0,2024-05-01 00:00:00.000000,1.0,0.0,2024-03-15 21:21:43.174528,exklusivně vám nabízím pronájem bytu 3+kk o ve...,3+kk,2,5. podlaží z celkem 5,...,None,None,31000,None,"Měsíční platba za služby pro 2 osoby je 2.250,...",Velmi dobrý,0,2,2024-03-15 21:21:43.174528,https://www.sreality.cz/detail/pronajem/byt/3+...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3944224076,"Trojická, Praha 2 - Nové Město",71.0,2024-04-01 00:00:00.000000,0.0,0.0,2024-03-16 11:57:33.477657,k pronájmu nabízíme nezařízený byt 3+1 o výměř...,3+1,2,2. podlaží z celkem 7,...,None,None,22000,None,+ domovní služby cca 850 Kč / osoba + převod e...,Velmi dobrý,0,2,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/3+...
35132748,"Řepská, Praha 6 - Ruzyně",80.0,2024-03-18 00:00:00.000000,0.0,0.0,2024-03-16 11:57:33.477657,"nabízím k dlouhodobému pronájmu vybavený, slun...",2+1,2,2. podlaží z celkem 3,...,None,None,24500,None,None,Velmi dobrý,0,2,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/2+...
1813517644,"Hradešínská, Praha 10 - Vinohrady",50.0,2024-03-18 00:00:00.000000,0.0,0.0,2024-03-16 11:57:33.477657,"nabízíme k pronájmu krásný, světlý, slunný byt...",2+kk,2,1. podlaží z celkem 4,...,None,None,20000,None,None,Po rekonstrukci,0,2,2024-03-16 11:57:33.477657,https://www.sreality.cz/detail/pronajem/byt/2+...


In [162]:
df[df['elevator'] == 'Výtah']

,address,area,available_from,balcony,cellar,created,description,disposition,elevator,floor,...,pets,public_transport,rent,security_deposit,service_fees,status,terrace,type,updated,url
id,,,,,,,,,,,,,,,,,,,,,
828380,"Grafická, Praha - Smíchov",31.0,2024-03-25,0.0,1.0,2024-03-15 21:21:43.174528,1+kk v bytovém komplexu neugraf.,1+kk,Výtah,1,...,None,MHD 2 minuty pěšky,17 850 Kč,25 000 Kč,2 834 Kč,Novostavba,Terasa 400 m²,Cihla,2024-03-15 21:21:43.174528,https://www.bezrealitky.cz/nemovitosti-byty-do...
720702,"Pod Viaduktem, Praha - Stodůlky",37.0,2024-06-01,0.0,1.0,2024-03-15 21:21:43.174528,nabízíme světlý prostorný byt 1+kk (37 m²) s v...,1+kk,Výtah,2,...,None,MHD 2 minuty pěšky,17 800 Kč,36 000 Kč,2 700 Kč,Novostavba,Terasa 12 m²,Ostatní,2024-03-15 21:21:43.174528,https://www.bezrealitky.cz/nemovitosti-byty-do...
784142,"Anny Drabíkové, Praha - Háje",42.0,2024-06-22,0.0,0.0,2024-03-15 21:21:43.174528,vzhledem k velmi vysokému zájmu o námi nabízen...,2+kk,Výtah,12,...,None,MHD 1 minuta pěšky,17 490 Kč,20 215 Kč,2 725 Kč,Po rekonstrukci,None,Panel,2024-03-15 21:21:43.174528,https://www.bezrealitky.cz/nemovitosti-byty-do...
830987,"Ruská, Praha - Vršovice",160.0,2024-05-13,0.0,0.0,2024-03-15 21:21:43.174528,útulná a tichá soukromá ložnice. velmi klidná ...,4+1,Výtah,5,...,None,MHD 2 minuty pěšky,17 617 Kč,None,5 872 Kč,None,None,None,2024-03-15 21:21:43.174528,https://www.bezrealitky.cz/nemovitosti-byty-do...
720398,"Bubenská, Praha - Holešovice",37.0,2024-04-01,0.0,0.0,2024-03-15 21:21:43.174528,zrekonstruovaný byt v domě po kompletní revita...,1+1,Výtah,3,...,None,MHD 2 minuty pěšky,16 500 Kč,20 000 Kč,None,Velmi dobrý,None,Cihla,2024-03-15 21:21:43.174528,https://www.bezrealitky.cz/nemovitosti-byty-do...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534927,"Rudolfa Antonína Dvorského, Praha - Horní Měch...",64.0,2024-03-16,0.0,0.0,2024-03-16 11:57:33.477657,"pronajmu prostorný, slunný, nezařízený byt 2+k...",2+kk,Výtah,3,...,None,MHD 5 minut pěšky,19 500 Kč,30 000 Kč,None,None,None,Cihla,2024-03-16 11:57:33.477657,https://www.bezrealitky.cz/nemovitosti-byty-do...
564171,"Na Maninách, Praha - Holešovice",48.0,2024-05-01,0.0,0.0,2024-03-16 11:57:33.477657,"pronajmu zařízený byt 2+kk, 50 m2 v 3. patře v...",2+1,Výtah,3,...,None,MHD 3 minuty pěšky,12 500 Kč,25 000 Kč,None,None,None,Cihla,2024-03-16 11:57:33.477657,https://www.bezrealitky.cz/nemovitosti-byty-do...
831332,"Strážnická, Praha - Čimice",44.0,2024-04-01,0.0,0.0,2024-03-16 11:57:33.477657,nabízím k dlouhodobému pronajmu byt 2+kk po ce...,2+kk,Výtah,4,...,None,MHD 2 minuty pěšky,16 000 Kč,32 000 Kč,4 190 Kč,Po rekonstrukci,None,Panel,2024-03-16 11:57:33.477657,https://www.bezrealitky.cz/nemovitosti-byty-do...


In [163]:
df.loc[:, 'elevator'] = df['elevator'].replace('Výtah.*', 1, regex=True)
df.loc[:, 'elevator'] = df['elevator'].replace(np.nan, 0)
df.loc[:, 'elevator'] = df['elevator'].replace(2, 0)
df.loc[:, 'elevator'].unique()

/tmp/ipykernel_21374/1434899049.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:, 'elevator'] = df['elevator'].replace('Výtah.*', 1, regex=True)
/tmp/ipykernel_21374/1434899049.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:, 'elevator'] = df['elevator'].replace(np.nan, 0)
/tmp/ipykernel_21374/1434899049.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to th

array([1.0, 0.0], dtype=object)

In [164]:
df.loc[:, 'floor'].sort_values().unique()
df.loc[:, 'floor'] = df['floor'].replace('. podlaží.*', '', regex=True)
df.loc[:, 'floor'] = df['floor'].replace(' z celkem.*', '', regex=True)
df.loc[:, 'floor'] = df['floor'].replace('přízemí', '0', regex=True).astype(int, errors='ignore')
df.loc[:, 'floor'].sort_values().unique()

array(['-1', '-2', '0', '1', '10', '11', '12', '13', '136', '14', '15',
       '17', '19', '2', '20', '21', '3', '4', '5', '6', '7', '8', '9',
       None], dtype=object)

In [165]:
df['front_garden'].unique()

array([None, 'Předzahrádka 8 m²', 'Předzahrádka 40 m²',
       'Předzahrádka 30 m²', 'Předzahrádka 10 m²', 'Předzahrádka 150 m²',
       'Předzahrádka 115 m²', 'Předzahrádka 16 m²', 'Předzahrádka 20 m²',
       'Předzahrádka 34 m²', 'Předzahrádka 520 m²',
       'Předzahrádka 4\xa0000 m²', 'Předzahrádka 72 m²',
       'Předzahrádka 15 m²', 'Předzahrádka 700 m²', 'Předzahrádka 42 m²',
       'Předzahrádka 40,1 m²', 'Předzahrádka 200 m²',
       'Předzahrádka 300 m²', 'Předzahrádka 100 m²',
       'Předzahrádka 117 m²', 'Předzahrádka 98 m²', 'Předzahrádka 500 m²',
       'Předzahrádka 1 m²', 'Předzahrádka 6 m²', 'Předzahrádka 27 m²',
       'Předzahrádka 50 m²', 'Předzahrádka 81 m²', 'Předzahrádka 26 m²',
       'Předzahrádka 80 m²', 'Předzahrádka 116 m²', 'Předzahrádka 18 m²',
       'Předzahrádka 400 m²', 'Předzahrádka 25 m²', 'Předzahrádka 7 m²',
       'Předzahrádka 23 m²', 'Předzahrádka 4 m²'], dtype=object)

In [166]:
df.loc[:, 'front_garden'] = df['front_garden'].replace('Předzahrádka .*', 1, regex=True)
df.loc[:, 'front_garden'].unique()

/tmp/ipykernel_21374/1031915580.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:, 'front_garden'] = df['front_garden'].replace('Předzahrádka .*', 1, regex=True)


array([nan, 1.0], dtype=object)

In [167]:
df['furnished'].unique()

array([0, 1, 3, 2, 'Částečně', 'Vybaveno', 'Nevybaveno', None],
      dtype=object)

In [168]:
df.loc[:, 'furnished'] = df['furnished'].replace('Nevybaveno', 'nevybaveny').replace('Částečně', 'castecne').replace('Vybaveno', 'vybaveny')
df.loc[:, 'furnished'] = df['furnished'].apply(lambda x: SrealityUrlBuilder.map_furnished_category(x) if isinstance(x, int) else x)



In [169]:
df['furnished'].unique()

array([None, 'vybaveny', 'castecne', 'nevybaveny'], dtype=object)

In [170]:
df.garage.unique()

array([0, 1, 'Garáž', None], dtype=object)

In [171]:
df.loc[:, 'garage'] = df['garage'].replace('Garáž.*', 1, regex=True)
df.garage.unique()

/tmp/ipykernel_21374/1718790291.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:, 'garage'] = df['garage'].replace('Garáž.*', 1, regex=True)


array([0.0, 1.0, nan], dtype=object)

In [172]:
df.loc[:, 'loggie'] = df['garage'].replace('Lodžie.*', 1, regex=True)
df.loggie.unique()

/tmp/ipykernel_21374/3599377027.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:, 'loggie'] = df['garage'].replace('Lodžie.*', 1, regex=True)


array([0.0, 1.0, nan], dtype=object)

In [173]:
df.columns

Index(['address', 'area', 'available_from', 'balcony', 'cellar', 'created',
       'description', 'disposition', 'elevator', 'floor', 'front_garden',
       'furnished', 'garage', 'gps_lat', 'gps_lon', 'last_seen', 'loggie',
       'parking', 'pets', 'public_transport', 'rent', 'security_deposit',
       'service_fees', 'status', 'terrace', 'type', 'updated', 'url'],
      dtype='object')

In [174]:
df.loc[:, 'parking'] = df['parking'].replace('Parkování.*', 1, regex=True)
df.parking.unique()


/tmp/ipykernel_21374/1487171255.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:, 'parking'] = df['parking'].replace('Parkování.*', 1, regex=True)


array([0.0, 1.0, nan], dtype=object)

In [175]:
df.loc[:, 'pets'] = df['pets'].replace('Domácí mazlíčci vítáni', 1, regex=True)
df.pets.unique()

/tmp/ipykernel_21374/3238766750.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:, 'pets'] = df['pets'].replace('Domácí mazlíčci vítáni', 1, regex=True)


array([nan, 1.0], dtype=object)

In [176]:
# drop public_transport column
df.drop(columns=['public_transport'], inplace=True)

/tmp/ipykernel_21374/629134586.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['public_transport'], inplace=True)


In [177]:
df.columns

Index(['address', 'area', 'available_from', 'balcony', 'cellar', 'created',
       'description', 'disposition', 'elevator', 'floor', 'front_garden',
       'furnished', 'garage', 'gps_lat', 'gps_lon', 'last_seen', 'loggie',
       'parking', 'pets', 'rent', 'security_deposit', 'service_fees', 'status',
       'terrace', 'type', 'updated', 'url'],
      dtype='object')

In [178]:
df.rent.unique()


array([19900, 13990, 25490, 22490, 12990, 37990, 17990, 19490, 16490,
       55000, 9500, 16500, 29500, 25000, 62888, 23000, 39900, 24880,
       24000, 40000, 12500, 17000, 15000, 31000, 14999, 70000, 12000,
       14900, 39000, 19800, None, 65403, 57857, 28490, 100000, 60270,
       60000, 120000, 30000, 88000, 36000, 59600, 57700, 33000, 55341,
       99000, 32000, 34990, 38000, 49000, 29000, 31444, 20000, 14000,
       22500, 44000, 37000, 80000, 15500, 38400, 62400, 42000, 27000,
       26000, '17 850 Kč', '17 800 Kč', '17 490 Kč', '17 617 Kč',
       '16 500 Kč', '17 000 Kč', '15 000 Kč', '15 500 Kč', 8870,
       '15 990 Kč', '15 900 Kč', '15 840 Kč', '15 834 Kč', '15 800 Kč',
       '15 700 Kč', '15 600 Kč', 19990, 13000, 61461, 28000, 72000,
       '15 490 Kč', '15 459 Kč', '14 990 Kč', 43000, '14 900 Kč',
       '14 753 Kč', '14 700 Kč', '14 500 Kč', '15 300 Kč', '14 490 Kč',
       '14 205 Kč', 47000, '14 000 Kč', 50000, 98000, '15 200 Kč', 10694,
       '16 990 Kč', '16 900

In [179]:
df.loc[:, 'rent'] = df['rent'].apply(lambda x: int(x.replace(' ', '').replace('Kč', '').replace('€', '')) if isinstance(x, str) else x)

df['rent'].sort_values().unique()

array([3000.0, 4000.0, 5000.0, 5100.0, 5300.0, 5500.0, 6000.0, 6250.0,
       6400.0, 6500.0, 6700.0, 6711.0, 7000.0, 7200.0, 7500.0, 7667.0,
       7900.0, 8000.0, 8500.0, 8700.0, 8870.0, 8900.0, 8990.0, 9000.0,
       9310.0, 9358.0, 9393.0, 9400.0, 9500.0, 9528.0, 9600.0, 9800.0,
       9900.0, 9990.0, 10000.0, 10013.0, 10158.0, 10400.0, 10490.0,
       10500.0, 10694.0, 10900.0, 10990.0, 11000.0, 11400.0, 11410.0,
       11490.0, 11500.0, 11600.0, 11800.0, 11900.0, 11990.0, 12000.0,
       12100.0, 12200.0, 12300.0, 12390.0, 12499.0, 12500.0, 12600.0,
       12690.0, 12700.0, 12745.0, 12800.0, 12870.0, 12872.0, 12880.0,
       12900.0, 12990.0, 13000.0, 13190.0, 13200.0, 13300.0, 13400.0,
       13404.0, 13490.0, 13500.0, 13575.0, 13600.0, 13700.0, 13748.0,
       13800.0, 13855.0, 13890.0, 13900.0, 13990.0, 13998.0, 13999.0,
       14000.0, 14001.0, 14004.0, 14007.0, 14205.0, 14250.0, 14290.0,
       14300.0, 14490.0, 14500.0, 14600.0, 14700.0, 14750.0, 14753.0,
       14800.0, 14

In [180]:
# 'security_deposit'

# print unique security_deposit values for rows wher url contains sreality
df.loc[df['url'].str.contains('bezrealitky')]['security_deposit'].unique()

array(['25 000 Kč', None, '36 000 Kč', '20 215 Kč', '20 000 Kč',
       '35 000 Kč', '15 000 Kč', '19 000 Kč', '17 400 Kč', '30 000 Kč',
       '18 000 Kč', '23 000 Kč', '18 132 Kč', '15 900 Kč', '17 000 Kč',
       '31 000 Kč', '30 999 Kč', '15 500 Kč', '18 417 Kč', '17 394 Kč',
       '30 918 Kč', '18 500 Kč', '17 500 Kč', '18 204 Kč', '17 662 Kč',
       '29 800 Kč', '29 600 Kč', '14 500 Kč', '27 000 Kč', '37 310 Kč',
       '12 000 Kč', '19 714 Kč', '22 000 Kč', '17 554 Kč', '21 000 Kč',
       '16 000 Kč', '28 000 Kč', '14 000 Kč', '16 500 Kč', '13 998 Kč',
       '23 772 Kč', '17 830 Kč', '34 000 Kč', '18 148 Kč', '20 095 Kč',
       '20 640 Kč', '33 614 Kč', '17 200 Kč', '33 000 Kč', '32 000 Kč',
       '120 000 Kč', '48 000 Kč', '19 300 Kč', '20 999 Kč', '148 000 Kč',
       '24 214 Kč', '19 135 Kč', '45 000 Kč', '10 000 Kč', '40 000 Kč',
       '38 230 Kč', '17 280 Kč', '18 889 Kč', '20 591 Kč', '18 972 Kč',
       '18 540 Kč', '24 000 Kč', '22 600 Kč', '19 515 Kč', '18 844 Kč

In [181]:
# drop security_deposit column
df.drop(columns=['security_deposit'], inplace=True)

/tmp/ipykernel_21374/2403586681.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['security_deposit'], inplace=True)


In [182]:
# 'service_fees', 'status', 'terrace', 'type', 'updated', 'url'

# unique values for service_fees
df.service_fees

id
660215116                                                  None
2543723852    + poplatky za služby vč. elektřiny 3.633 Kč/měsíc
3952227660    + poplatky 3.550 Kč/měsíc + elektřina, která b...
3895641420    +poplatky 3.080 Kč/měsíc + elektřina a plyn, k...
498279756     + poplatky 1.822 Kč/měsíc + elektřina, která b...
                                    ...                        
1998067020                         + 2500 zúčtovatelné poplatky
2534937932      + poplatky 2.600,- Kč/měs. + spotřeba elektřiny
3320321356      + poplatky vč. energií a internetu 5.500Kč/měs.
3085440332                                + 1500 zálohy energie
3286766924                                                 None
Name: service_fees, Length: 5278, dtype: object

In [183]:
df.drop(columns=['service_fees'], inplace=True)

/tmp/ipykernel_21374/3995940248.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['service_fees'], inplace=True)


In [184]:
# 'status'

# unique values for status
df.status.unique()

array(['Po rekonstrukci', 'Velmi dobrý', 'Dobrý', 'Novostavba', None,
       'Ve výstavbě', 'Projekt', 'V rekonstrukci', 'Před rekonstrukcí'],
      dtype=object)

In [185]:
# 'terrace', 
df.loc[:, 'terrace'] = df['terrace'].apply(lambda x: 1 if isinstance(x, str) and "Terasa" in x else x)
df.loc[:, 'terrace'].unique()

array([0.0, 1.0, nan], dtype=object)

In [186]:
# 'type'
df.type.unique()
df.loc[:, 'type'] = df['type'].apply(lambda x: x.replace('Cihla', 'cihlova').replace('Panel', 'panelova').replace('Smíšená', 'ostatni').replace('Skeletová', 'ostatni').replace('Nízkoenergetická', 'ostatni').replace('Montovaná', 'ostatni').replace('Dřevostavba', 'ostatni').replace('Kamenná', 'ostatni').replace('Ostatní', 'ostatni') if isinstance(x, str) else x)
df.loc[:, 'type'] = df['type'].apply(lambda x: SrealityUrlBuilder.map_building_type(x) if isinstance(x, int) else x)
df.type.unique()

array(['ostatni', 'cihlova', 'panelova', None], dtype=object)

In [187]:
# save df to filtered a sqlite database without using the databasewrapper
import sqlite3
conn = sqlite3.connect('filtered_listings.db')
df.to_sql('listings', conn, if_exists='replace')
conn.close()